## Pitch Names Classifier

In [1]:
import numpy as np

from htm.bindings.sdr import SDR
from htm.bindings.encoders import ScalarEncoder, ScalarEncoderParameters
from htm.algorithms import SpatialPooler as SP
from htm.algorithms import TemporalMemory as TM
from htm.bindings.algorithms import Classifier

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

#import pickle

In [2]:
pitchNames = ("C","D","E","F","G","A","B")

scalarEncoderParams = ScalarEncoderParameters()
scalarEncoderParams.minimum = 0
scalarEncoderParams.maximum = 6
scalarEncoderParams.activeBits = 3
scalarEncoderParams.category = True

enc = ScalarEncoder(scalarEncoderParams)

print ("C: ", enc.encode(0))
print ("D: ", enc.encode(1))
print ("E: ", enc.encode(2))
print ("F: ", enc.encode(3))
print ("G: ", enc.encode(4))
print ("A: ", enc.encode(5))
print ("B: ", enc.encode(6))

C:  SDR( 21 ) 0, 1, 2
D:  SDR( 21 ) 3, 4, 5
E:  SDR( 21 ) 6, 7, 8
F:  SDR( 21 ) 9, 10, 11
G:  SDR( 21 ) 12, 13, 14
A:  SDR( 21 ) 15, 16, 17
B:  SDR( 21 ) 18, 19, 20


In [3]:
inputSDR  = SDR( dimensions = (21, ) )
activeSDR = SDR( dimensions = (576,) )
sp = SP(inputDimensions  = inputSDR.dimensions,
        columnDimensions = activeSDR.dimensions,
        localAreaDensity = 0.02,
        globalInhibition = True,
        seed             = 1,
        synPermActiveInc   = 0.01,
        synPermInactiveDec = 0.008)

print(sp)

Spatial Pooler Connections:
    Inputs (21) ~> Outputs (576) via Segments (576)
    Segments on Cell Min/Mean/Max 1 / 1 / 1
    Potential Synapses on Segment Min/Mean/Max 11 / 11 / 11
    Connected Synapses on Segment Min/Mean/Max 1 / 5.44097 / 10
    Synapses Dead (0%) Saturated (0%)
    Synapses pruned (0%) Segments pruned (0%)



In [4]:
clsr = Classifier()

In [5]:
tm = TM(
    columnDimensions = (576,),
    cellsPerColumn=8,
    initialPermanence=0.5,
    connectedPermanence=0.5,
    minThreshold=8,
    maxNewSynapseCount=20,
    permanenceIncrement=0.1,
    permanenceDecrement=0.0,
    activationThreshold=8,
)
print(tm)

Temporal Memory Connections:
    Inputs (0) ~> Outputs (4608) via Segments (0)
    Segments on Cell Min/Mean/Max 0 / 0 / 0
    Potential Synapses on Segment Min/Mean/Max 4294967295 / nan / 0
    Connected Synapses on Segment Min/Mean/Max 65535 / nan / 0
    Synapses Dead (nan%) Saturated (nan%)
    Synapses pruned (nan%) Segments pruned (nan%)



In [6]:
for i in range(len(pitchNames)):
    inputSDR = enc.encode(i)
    print("input SDR: ",inputSDR)
    sp.compute(inputSDR, True, activeSDR)
    print("Active SDR: ",activeSDR)
    clsr.learn( activeSDR.addNoise(0.2), i )
    print("Classifier learn: ",i)
    print("")

input SDR:  SDR( 21 ) 0, 1, 2
Active SDR:  SDR( 576 ) 62, 113, 121, 222, 243, 320, 409, 453, 478, 567, 572
Classifier learn:  0

input SDR:  SDR( 21 ) 3, 4, 5
Active SDR:  SDR( 576 ) 99, 152, 297, 372, 485, 526, 537, 552, 555, 564, 575
Classifier learn:  1

input SDR:  SDR( 21 ) 6, 7, 8
Active SDR:  SDR( 576 ) 67, 103, 183, 440, 542, 545, 546, 547, 558, 571, 573
Classifier learn:  2

input SDR:  SDR( 21 ) 9, 10, 11
Active SDR:  SDR( 576 ) 55, 174, 254, 286, 313, 340, 370, 414, 513, 544, 565
Classifier learn:  3

input SDR:  SDR( 21 ) 12, 13, 14
Active SDR:  SDR( 576 ) 39, 60, 69, 211, 382, 509, 523, 540, 552, 556, 573
Classifier learn:  4

input SDR:  SDR( 21 ) 15, 16, 17
Active SDR:  SDR( 576 ) 71, 120, 169, 206, 218, 271, 312, 313, 336, 421, 573
Classifier learn:  5

input SDR:  SDR( 21 ) 18, 19, 20
Active SDR:  SDR( 576 ) 47, 95, 102, 112, 205, 247, 310, 362, 413, 500, 575
Classifier learn:  6



In [7]:
#seq = ["C","C","G","G","A","A","G","F","F","E","E","D","D","C"]
seq=[0,0,4,4,5,5,4,3,3,2,2,1,1,0]

#seq_clsr = Classifier()

In [8]:
#batch=10
#for n in range(batch):    
#    for i in range(len(seq)):
#        inputSDR = enc.encode(seq[i])
#        print("input SDR: ",inputSDR)
#        sp.compute(inputSDR, True, activeSDR)
#        print("Active SDR: ",activeSDR)
#        print("")

#        tm.compute( activeSDR, learn=True)
#        tm.activateDendrites(True)

#        activeColumnsIndices   = [tm.columnForCell(i) for i in tm.getActiveCells().sparse]
#        predictedColumnIndices = [tm.columnForCell(i) for i in tm.getPredictiveCells().sparse]

#        print(tm.getActiveCells())
 #       print(tm.getPredictiveCells())

 #       active_sdr = SDR( tm.numberOfColumns() )
#        active_sdr.sparse  = np.array(sorted(set(activeColumnsIndices)))
 #       print("-"*70 )
#        print("Active sdr: ", active_sdr)

#        predict_sdr = SDR( tm.numberOfColumns() )
#        predict_sdr.sparse  = np.array(sorted(set(predictedColumnIndices)))
#        print("-"*70 )
#        print("Predicted sdr: ",predict_sdr)

#        if  len(predict_sdr.sparse) == 0:
#            tm_predict = "nan"
#            tm_pitchNames = "nan"      
#        else:
#            tm_predict = np.argmax( clsr.infer( predict_sdr) ) 
#            tm_pitchNames = pitchNames[tm_predict]

#        print("-"*70 )
#        print("PDF: ",clsr.infer( predict_sdr) )
#        print("-"*70 )

#        print("predict index: ",tm_predict)
#        print("-"*70 )
#        print("predict label: ",tm_pitchNames)

#        print("")
        
#tm.saveToFile("tm_saved.model")

In [10]:
for i in range(len(seq)):
    print("--- ", i ," ---")
    inputSDR = enc.encode(seq[i])
    print("input SDR: ",inputSDR)
    sp.compute(inputSDR, True, activeSDR)
    print("Active SDR: ",activeSDR)
    print("")

    tm.loadFromFile("tm_saved.model")
    print(tm)
        
    tm.compute( activeSDR, learn=False)
    tm.activateDendrites(True)
    activeColumnsIndices   = [tm.columnForCell(i) for i in tm.getActiveCells().sparse]
    predictedColumnIndices = [tm.columnForCell(i) for i in tm.getPredictiveCells().sparse]

    print(tm.getActiveCells())
    print(tm.getPredictiveCells())

    active_sdr = SDR( tm.numberOfColumns() )
    active_sdr.sparse  = np.array(sorted(set(activeColumnsIndices)))
    print("-"*70 )
    print("Active sdr: ", active_sdr)

    predict_sdr = SDR( tm.numberOfColumns() )
    predict_sdr.sparse  = np.array(sorted(set(predictedColumnIndices)))
    print("-"*70 )
    print("Predicted sdr: ",predict_sdr)

    if  len(predict_sdr.sparse) == 0:
        tm_predict = "nan"
        tm_pitchNames = "nan"      
    else:
        tm_predict = np.argmax( clsr.infer( predict_sdr) ) 
        tm_pitchNames = pitchNames[tm_predict]

    print("-"*70 )
    print("PDF: ",clsr.infer( predict_sdr) )
    print("-"*70 )

    print("predict index: ",tm_predict)
    print("-"*70 )
    print("predict label: ",tm_pitchNames)

    print("")

---  0  ---
input SDR:  SDR( 21 ) 0, 1, 2
Active SDR:  SDR( 576 ) 62, 113, 121, 222, 243, 320, 409, 453, 478, 567, 572

Temporal Memory Connections:
    Inputs (143) ~> Outputs (4608) via Segments (143)
    Segments on Cell Min/Mean/Max 0 / 0.031033 / 1
    Potential Synapses on Segment Min/Mean/Max 11 / 12.1469 / 22
    Connected Synapses on Segment Min/Mean/Max 11 / 12.1469 / 22
    Synapses Dead (0%) Saturated (0.975245%)
    Synapses pruned (0%) Segments pruned (0%)

SDR( 576, 8 ) 496, 904, 968, 1776, 1944, 2560, 3272, 3624, 3824, 4536, 4576
SDR( 576, 8 ) 312, 480, 496, 552, 904, 968, 1688, 1776, 1944, 2560, 3056, 3272, 3624, 3824, 4072, 4184, 4320, 4416, 4448, 4536, 4576, 4584
----------------------------------------------------------------------
Active sdr:  SDR( 576 ) 62, 113, 121, 222, 243, 320, 409, 453, 478, 567, 572
----------------------------------------------------------------------
Predicted sdr:  SDR( 576 ) 39, 60, 62, 69, 113, 121, 211, 222, 243, 320, 382, 409, 453, 47

In [ ]:
["C","C","G","G","A","A","G","F","F","E","E","D","D","C"]

In [ ]:
["C","G","G","G","A","A","G","F","F","E","E","D","D","C"]